In [ ]:
!pip install docx2txt

In [ ]:
!pip install sentence_transformers

In [ ]:
!pip install docx

In [ ]:
!pip install PyPDF2

In [ ]:
!pip install docx2txt


In [42]:
#NECESSARY MODULE
import os
import pandas as pd
from sentence_transformers import SentenceTransformer, util
from PyPDF2 import PdfReader
import docx2txt

#READING PDF
def extract_text_from_document(file_path):
    text = ""
    if file_path.endswith('.pdf'):
        try:
            with open(file_path, 'rb') as file:
                reader = PdfReader(file)
                for page in reader.pages:
                    text += page.extract_text()
        except Exception as e:
            print(f"Error reading PDF file: {e}")
    elif file_path.endswith('.docx'):
        try:
            text = docx2txt.process(file_path)
        except Exception as e:
            print(f"Error reading DOCX file: {e}")
    else:
        print("Unsupported file format")
    return text

#RANK
def rank_resumes_sentence_transformers(resume_directory, job_description_path, model_name):
    job_description_text = extract_text_from_document(job_description_path)
    resume_files = [file for file in os.listdir(resume_directory)]

    model = SentenceTransformer(model_name)
    job_description_embedding = model.encode(job_description_text, convert_to_tensor=True)

    scores = []
    for file in resume_files:
        text = extract_text_from_document(os.path.join(resume_directory, file))
        text_embedding = model.encode(text, convert_to_tensor=True)


        similarity = float(util.pytorch_cos_sim(job_description_embedding, text_embedding)[0][0])

        scores.append(similarity)

    resume_data = pd.DataFrame({
        'Resume': resume_files,
        'Similarity_Score': scores
    })
    ranked_resumes = resume_data.sort_values(by='Similarity_Score', ascending=False)
    return ranked_resumes

#MODEL & FILE PATH
resume_directory = "/content/drive/MyDrive/PythonRes"
job_description_path = "/content/drive/MyDrive/RANKING/JOB_DES.pdf"
model_name = 'sentence-transformers/msmarco-MiniLM-L-12-v3'

ranked_resumes_sentence_transformers = rank_resumes_sentence_transformers(resume_directory, job_description_path, model_name)


ranked_resumes_sentence_transformers.to_csv("/content/drive/MyDrive/RANKING/ranked_resumes_sentence_transformers.csv", index=False)

print("Ranked Resumes by Similarity (Sentence Transformers):")
print(ranked_resumes_sentence_transformers[['Resume', 'Similarity_Score']])



Ranked Resumes by Similarity (Sentence Transformers):
                           Resume  Similarity_Score
3  Python Developer_resume_1.docx          0.813476
1  Python Developer_resume_3.docx          0.653458
0  Python Developer_resume_0.docx          0.633810
5  Python Developer_resume_7.docx          0.627064
6  Python Developer_resume_5.docx          0.528187
4  Python Developer_resume_2.docx          0.510891
2  Python Developer_resume_4.docx          0.434695
8  Python Developer_resume_8.docx          0.417042
7  Python Developer_resume_6.docx          0.340740
